Feature engineering, partie de Polina
(le code pour exploration de données de Polina est dans data_explore_polina.ipynb)

Prend en input:
Fichier de données issu du feature_engineering Denis+Vincent (engineerd1.csv)

Réalise:
les traitements de NaN et OHE 

Fournit en sortie:
Fichier de données prêt pour faire une standardisation (engineerd2.csv)

prochain traitement après celui-ci : scale.ipynb


In [1]:
%run init_notebook.py

In [2]:
# J'importe la base avec plusieurs colonnes déjà encodées
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(filepath_or_buffer = base_processed+"engineerd1_cat.csv")
df.head()

,index,Cn,m (kg),Ewltp (g/km),W (mm),At1 (mm),Fm,ec (cm3),ep (KW),z (Wh/km),...,IT37,IT38,IT39,ep (ch),Ft_diesel/electric,Ft_petrol,Ft_petrol/electric,Cr_M1G,Cr_M1S,Cr_N1G
0,2,MITSUBISHI ECLIPSE CROSS,1975.0,46.0,2670.0,1540.0,P,2360.0,72.0,175.0,...,0,0,0,97.92,0.0,0.0,1.0,0.0,0.0,0.0
1,4,TOYOTA AYGO X,1020.0,108.0,2430.0,1540.0,M,998.0,53.0,NaN,...,0,0,0,72.08,0.0,0.0,1.0,0.0,0.0,0.0
2,5,PANDA,1067.0,112.0,2300.0,1600.0,H,999.0,52.0,NaN,...,0,0,0,70.72,0.0,1.0,0.0,0.0,0.0,0.0
3,8,SPORTAGE,1905.0,25.0,2680.0,1615.0,P,1598.0,132.0,169.0,...,0,0,0,179.52,0.0,0.0,1.0,0.0,0.0,0.0
4,9,SPORTAGE,1905.0,25.0,2680.0,1615.0,P,1598.0,132.0,169.0,...,0,0,0,179.52,0.0,0.0,1.0,0.0,0.0,0.0


On supprime les lignes avec NAN dans la colonne Cylindrée

In [3]:
# nb de NaN avant la suppression
nb_nan = df["ec (cm3)"].isna().sum()

print("Nombre de NaN dans la colonne 'ec (cm3)' : ", nb_nan)

Nombre de NaN dans la colonne 'ec (cm3)' :  25


In [4]:
df = df.dropna(subset=['ec (cm3)'])

In [5]:
# vérif NaN après suppression
nb_nan = df["ec (cm3)"].isna().sum()

print(f"Nombre de NaN dans la colonne 'ec (cm3)' : {nb_nan}")

Nombre de NaN dans la colonne 'ec (cm3)' : 0


Traitement des NaN dans la colonne Electric Range

Ft_diesel/electric

In [6]:
# Filtrer le DataFrame pour Ft_diesel/electric == 1
df_filtered = df[df['Ft_diesel/electric'] == 1]

# Compter les NaN dans la colonne 'Electric range (km)'
nb_nan = df_filtered['Electric range (km)'].isna().sum()

print("Nombre de NaN dans 'Electric range (km)' après filtrage : ", nb_nan)

Nombre de NaN dans 'Electric range (km)' après filtrage :  2


In [7]:
df = df[~((df['Ft_diesel/electric'] == 1) & (df['Electric range (km)'].isna()))]

In [8]:
# Vérif NaN
df_filtered = df[df['Ft_diesel/electric'] == 1]

# Compter les NaN dans la colonne 'Electric range (km)'
nb_nan = df_filtered['Electric range (km)'].isna().sum()

print("Nombre de NaN dans 'Electric range (km)' après filtrage : ", nb_nan)

Nombre de NaN dans 'Electric range (km)' après filtrage :  0


Ft_petrol/electric

In [9]:
# Filtrer le DataFrame pour Ft_petrol/electric == 1
df_filtered = df[df['Ft_petrol/electric'] == 1]

# Compter les NaN dans la colonne 'Electric range (km)'
nb_nan = df_filtered['Electric range (km)'].isna().sum()

print("Nombre de NaN dans 'Electric range (km)' après filtrage : ", nb_nan)

Nombre de NaN dans 'Electric range (km)' après filtrage :  17199


In [10]:
df = df[~((df['Ft_petrol/electric'] == 1) & (df['Electric range (km)'].isna()))]

In [11]:
# Vérif NaN
df_filtered = df[df['Ft_petrol/electric'] == 1]

# Compter les NaN dans la colonne 'Electric range (km)'
nb_nan = df_filtered['Electric range (km)'].isna().sum()

print("Nombre de NaN dans 'Electric range (km)' après filtrage : ", nb_nan)

Nombre de NaN dans 'Electric range (km)' après filtrage :  0


Diesel : non Nan dans Electric range (km) >> Nan >> 0

In [12]:
# nombre de lignes où Ft_diesel == 1 et où Electric range (km) n'est pas NaN
count = df[
    (df['Ft_petrol'] == 0) &
    (df['Ft_petrol/electric'] == 0) &
    (df['Ft_diesel/electric'] == 0) &
    (df['Electric range (km)'].notna())
].shape[0]

print(count)

6


In [13]:
# transformer les non nan de diesel par nan
conditions = (
    (df['Ft_petrol'] == 0) &
    (df['Ft_petrol/electric'] == 0) &
    (df['Ft_diesel/electric'] == 0) &
    (df['Electric range (km)'].notna())
)

# Nombre de lignes correspondant aux conditions
nb_non_nan = df[conditions].shape[0]
print("Nombre de lignes avant modification : ", nb_non_nan)

# Remplacer les valeurs par NaN
df.loc[conditions, 'Electric range (km)'] = np.nan

Nombre de lignes avant modification :  6


In [14]:
# Vérifier les modifications appliquées

conditions = (
    (df['Ft_petrol'] == 0) &
    (df['Ft_petrol/electric'] == 0) &
    (df['Ft_diesel/electric'] == 0) &
    (df['Electric range (km)'].notna())
)

nb_non_nan_apres = df[conditions].shape[0]
print("Nombre de lignes après modification : ", nb_non_nan_apres)

Nombre de lignes après modification :  0


In [15]:
# On remplace tous les Nan par 0

conditions = (
    (df['Ft_petrol'] == 0) &
    (df['Ft_petrol/electric'] == 0) &
    (df['Ft_diesel/electric'] == 0) &
    (df['Electric range (km)'].isna())  # Vérifier que la colonne contient NaN
)

df.loc[conditions, 'Electric range (km)'] = 0


In [16]:
# On vérifie qu'il ne reste plus de NaN dans Electric range (km) pour Diesel
count = df[
    (df['Ft_petrol'] == 0) &
    (df['Ft_petrol/electric'] == 0) &
    (df['Ft_diesel/electric'] == 0) &
    (df['Electric range (km)'].isna())
].shape[0]

print("Nombre de lignes correspondantes : ", count)

Nombre de lignes correspondantes :  0


Traitement des NaN dans Electric range (km) pour Ft_petrol

In [17]:
# nb de lignes avec Nan dans Electric range (km) pour Ft_petrol avant traitement
count = df[(df['Ft_petrol'] == 1) & (df['Electric range (km)'].isna())].shape[0]
print("Nombre de lignes correspondantes : ", count)

Nombre de lignes correspondantes :  618311


In [18]:
# remplacement des nan par 0
condition = (df['Ft_petrol'] == 1) & (df['Electric range (km)'].isna())

# Remplacement des NaN par 0 pour les lignes correspondant à la condition
df.loc[condition, 'Electric range (km)'] = 0

In [44]:
# nb de lignes avec Nan dans Electric range (km) pour Ft_petrol après traitement
count = df[(df['Ft_petrol'] == 1) & (df['Electric range (km)'].isna())].shape[0]
print("Nombre de lignes correspondantes : ", count)

Nombre de lignes correspondantes :  0


Fuel mode (Fm) - variable nominative >>> One hot Encoding

In [19]:
# Nb de lignes par modalité de la colonne Fm avant OHE

counts = df['Fm'].value_counts(dropna=False)
print(counts)

Fm
M    835344
H    315505
P    171249
Name: count, dtype: int64


In [ ]:
# One Hot Encoding de la variable Fm

df = pd.get_dummies(df, columns=["Fm"], prefix="Fm")
df.head()

In [21]:
# au cas où le get_dummies a créé des True/False au lieu de 1/0

# Liste des colonnes à convertir
cols_to_convert = ["Fm_H", "Fm_M", "Fm_P"]

# Appliquer la conversion uniquement à ces colonnes
df[cols_to_convert] = df[cols_to_convert].astype(int)

In [23]:
# vérif du nb de Fm_M après OHE
counts = df['Fm_M'].value_counts(dropna=False)
print(counts)

Fm_M
1    835344
0    486754
Name: count, dtype: int64


***
Quelques traitements finaux

In [24]:
# Supprimons les colonnes "ep (ch)" et "Cn" qui ne nous ont servi que pour des besoins de compréhension humaine des données
# bien entendu nous devons aussi supprimer index qui n'est pas une valeur métier
df.drop(columns=["ep (ch)", "Cn", "index"], inplace=True)

In [25]:
# remplacer les NaN par zéro pour "z (Wh/km)"
df["z (Wh/km)"].fillna(0, inplace=True)

C:\Users\abcd3\AppData\Local\Temp\ipykernel_21200\1170920781.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["z (Wh/km)"].fillna(0, inplace=True)


In [26]:
nan_count = df["Ewltp (g/km)"].isna().sum()
print(f"Nombre de NaN dans notre variable cible 'Ewltp (g/km)': {nan_count}, on supprime ces lignes")

Nombre de NaN dans notre variable cible 'Ewltp (g/km)': 458, on supprime ces lignes


In [27]:
df.dropna(subset=["Ewltp (g/km)"], inplace=True)

In [28]:
# enfin, après toutes nos feature engineering, et suppressions de colonnes, il y a maintenant des duplicats,
# on les supprime
df.drop_duplicates(inplace=True)


In [29]:
# Sauvegarde des données après les modifications
df.to_csv(base_processed + 'engineerd2_cat.csv', index=False)